### PLEASE COPY NOTEBOOKS TO YOUR FOLDERS TO PREVENT COMMIT CONFLICTS
* If you would like to contribute to this notebook, make changes on it in useful_notebooks folder, run "Restart and Clear Output" before commit.

#### Unexpected Expansion Cases
###### Experiment set fetches unrealated labs and awards
Sometimes a fetch will get some unrelated labs and awards, this is because of the multiple awards a lab can have. This multiple awards are visited, which have users linked to them. This users also have labs, so here you go. Hopefully all are released/current already.

###### Experiment set fetches unrealted biosmaple/experiment/set
This was so far because of the experiment and biosample relation field, or the references field that links to a publication. If you ignore these fields, if should be fine.
`ignore_field = ['experiment_relation', 'biosample_relation', 'references']`

In [2]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time


#my_auth = get_key('andyprod', keyfile='~/saved_keypairs.json')
my_auth = get_key('koray_data')

# Please Modify the following accordingly 
# do you want to check for duplicate/problematic runs on files?
# it will take some time

search_url  = '/search/?experiments_in_set.experiment_type.display_title=in+situ+Hi-C&experimentset_type=replicate&type=ExperimentSetReplicate&status=released&status=released to project'
set_to_release = ff_utils.search_metadata(search_url, my_auth)
print(len(set_to_release))
for a_set in set_to_release: 
    time1 = time.time()
    store, uuids = ff_utils.expand_es_metadata([a_set['uuid']], my_auth, store_frame='embedded',add_pc_wfr=True, ignore_field = ['experiment_relation', 'biosample_relation', 'references', 'reference_pubs'])
    time2 = time.time()
    diff = round((time2-time1), 1)
    print(a_set['accession'], len(uuids), 'items collected in', diff)
    analyze(store, True, False, a_set['status'])

67
4DNES76KXUJ3 243 items collected in 3.2
4DNESNLXBFMY 151 items collected in 2.9
4DNESIU6F8HF 149 items collected in 2.4
4DNESSQU7B76 151 items collected in 2.9
4DNESFOADERB 135 items collected in 3.2
file_reference            Out of 3, 0 skipped, 1 matching, 0 unusual, and 2 needs change
file_processed            Out of 10, 0 skipped, 7 matching, 0 unusual, and 3 needs change
workflow_run_awsem        Out of 16, 0 skipped, 3 matching, 0 unusual, and 13 needs change
4DNESFI64TG3 136 items collected in 2.5
file_reference            Out of 3, 0 skipped, 1 matching, 0 unusual, and 2 needs change
file_processed            Out of 10, 0 skipped, 7 matching, 0 unusual, and 3 needs change
workflow_run_awsem        Out of 16, 0 skipped, 3 matching, 0 unusual, and 13 needs change
4DNESU95RUNO 141 items collected in 2.1
file_processed            Out of 10, 0 skipped, 7 matching, 0 unusual, and 3 needs change
workflow_run_awsem        Out of 16, 0 skipped, 3 matching, 0 unusual, and 13 needs cha

4DNESEYETNMX 121 items collected in 5.0
higlass_view_config       Out of 2, 0 skipped, 0 matching, 0 unusual, and 2 needs change
file_processed            Out of 6, 0 skipped, 3 matching, 0 unusual, and 3 needs change
workflow_run_awsem        Out of 5, 0 skipped, 2 matching, 0 unusual, and 3 needs change
quality_metric_pairsqc    Out of 2, 0 skipped, 1 matching, 0 unusual, and 1 needs change
4DNESX29AMHF 121 items collected in 6.0
higlass_view_config       Out of 2, 0 skipped, 0 matching, 0 unusual, and 2 needs change
file_processed            Out of 6, 0 skipped, 3 matching, 0 unusual, and 3 needs change
workflow_run_awsem        Out of 5, 0 skipped, 2 matching, 0 unusual, and 3 needs change
quality_metric_pairsqc    Out of 2, 0 skipped, 1 matching, 0 unusual, and 1 needs change
4DNESAZ12B8V 121 items collected in 6.5
higlass_view_config       Out of 2, 0 skipped, 0 matching, 0 unusual, and 2 needs change
file_processed            Out of 6, 0 skipped, 3 matching, 0 unusual, and 3 nee

4DNESCIHJOXA 142 items collected in 2.9


In [1]:
# TODO
# Check audits
def analyze(store, check_wfrs, delete_problematic, change_status):
    # create stash of wfrs to pass to delete_wfrs
    stash = store.get('workflow_run_sbg', []) + store.get('workflow_run_awsem', [])

    # to decide which items to ignore as they have a 'higher' status
    STATUS_LEVEL = {
        # standard_status
        "released": 10, "current": 10,
        "released to project": 9,
        "pre-release": 8,
        "planned": 6, "submission in progress": 6,
        "in review by lab": 4,
        "revoked": 0, "archived": 0,"deleted": 0, "obsolete": 0, "replaced": 0, "archived to project": 0,
        # additional file statuses
        'to be uploaded by workflow': 4, 'uploading': 4, 'uploaded': 4, 'upload failed': 4, 'draft': 4, 'released to lab': 4}

    for a_set in store['experiment_set_replicate']:
        if not a_set.get('completed_processes'):
            print(a_set['accession'], 'missing processing tag', a_set['description'][:50])

    # check for experiment numbers
    exp_names = [i for i in store if i.startswith('experiment') and i not in ['experiment_set', 'experiment_set_replicate', 'experiment_type']]
    all_exps_on_sets = [a for i in store['experiment_set_replicate'] for a in i['experiments_in_set']]
    all_exps = [a['uuid'] for i in store.keys() for a in store[i] if i in exp_names]
    if len(all_exps_on_sets) != len(all_exps):
        print('Number of experiments is not same as experiments associated with sets')
        print('# of exps: {}. # of exps on sets: {}'.format(len(all_exps), len(all_exps_on_sets)))

    for a_file in store['file_fastq']:
        if not a_file.get('quality_metric'):
            print(a_file['accession'], 'missing fastqc')
        if not a_file.get('content_md5sum'):
            print(a_file['accession'], 'missing content md5 sum')
        if not a_file.get('md5sum'):
            print(a_file['accession'], 'md5 was not calculated during upload, missing md5sum')
        if check_wfrs:
            dw = delete_wfrs(a_file, my_auth, delete=delete_problematic, stash=stash)

    if store.get('file_processed'):
        for a_file in store['file_processed']:
            if a_file['file_format']['file_format'] == '/file-formats/pairs/':
                if not a_file.get('quality_metric'):
                    print(a_file['accession'], 'missing Pairsqc')
            if not a_file.get('source_experiments'):
                print(a_file['accession'], 'user submitted or produced by sbg runs')
            if check_wfrs:
                dw = delete_wfrs(a_file, my_auth, delete=delete_problematic, stash=stash)   


    if store.get('workflow_run_awsem'):
        for wfr in store['workflow_run_awsem']:
            if wfr['run_status'] != 'complete':
                print('problematic wfr', wfr['uuid'], wfr['run_status'])


    change_level = STATUS_LEVEL.get(change_status, 1)
    
    print_each = False
    counter = 0
    for a_type in store:
        total = len(store[a_type])
        change = 0
        matching = 0
        unusual = 0
        skipping = 0
        for raw_data in store[a_type]:
            if STATUS_LEVEL.get(raw_data['status']) > change_level:
                skipping += 1
                msg = ('{} {} ITEM HAS STATUS {} HIGHER THAN {} - SKIPPING'.format(a_type, raw_data['uuid'], raw_data['status'], change_status))
            elif STATUS_LEVEL.get(raw_data['status']) == STATUS_LEVEL.get(change_status):
                matching += 1
                msg = ('MATCHING ACCESS STATUS', a_type, raw_data['uuid'], raw_data['status'])
            elif STATUS_LEVEL.get(raw_data['status']) == 0:
                unusual += 1
                msg = ('SKIP UNUSUAL STATUS  ', a_type, raw_data['uuid'], raw_data['status'])
            else:
                change += 1
                msg = ('       CHANGE        ', a_type, raw_data['uuid'], raw_data['status'])
            if print_each:
                print(msg)
        if change>0:
            print('{:<25} Out of {t}, {r} skipped, {m} matching, {u} unusual, and {c} needs change'.format(a_type, t=total, r=skipping, m=matching, u=unusual, c=change))



In [39]:
# # clear opf
# for exp in  store.get('experiment_tsaseq'):
#     ff_utils.patch_metadata({}, exp['uuid'], my_auth, add_on='delete_fields=other_processed_files')
#     pass
# # clear tag
# for aset in store.get('experiment_set_replicate'):
#     ff_utils.patch_metadata({}, aset['uuid'], my_auth, add_on='delete_fields=completed_processes')
#     pass

# def fetch_wfr_associated(wfr_uuid, my_key):
#     """Given wfr_uuid, find associated output files and qcs"""
#     wfr_as_list = []
#     wfr_info = ff_utils.get_metadata(wfr_uuid, my_key)
#     print(wfr_info['display_title'])
#     wfr_as_list.append(wfr_info['uuid'])
#     if wfr_info.get('output_files'):
#         for o in wfr_info['output_files']:
#                 if o.get('value'):
#                     wfr_as_list.append(o['value']['uuid'])
#                 if o.get('value_qc'):
#                     wfr_as_list.append(o['value_qc']['uuid']) # this is a @id
#     if wfr_info.get('output_quality_metrics'):
#         for qc in wfr_info['output_quality_metrics']:
#             if qc.get('value'):
#                 wfr_as_list.append(qc['value']['uuid'])
#     return wfr_as_list

# for wfr in store['workflow_run_awsem']:
#     if wfr['display_title'].startswith('repliseq'):
#         lip = fetch_wfr_associated(wfr['uuid'], my_auth)
#         print(lip)
#         for uu in lip:
#             ff_utils.patch_metadata({'status': 'deleted'}, uu, my_auth)
#             pass
        

repliseq-parta v16 run 2019-03-12 19:49:16.023049
['e10765e2-ebaf-40c8-ac7d-a9ce006fff0b', '31a25232-884f-48fc-979b-463eb6e8a2dd', 'd2745e28-8536-450b-9fbb-e9d0ff92fbed', '47e18ae7-a426-4f00-9e6d-b69234850aa4']
repliseq-parta v16 run 2019-03-12 19:49:22.994268
['b99b7355-58ab-4c3f-8f99-e4058acee00e', '12593b06-d2c0-40ac-989a-64ff934f9f71', '4670ad68-25e1-4401-9744-733d9fd8d991', '2f89c0f7-f8ae-4f97-bd46-65ffc03f2a3a']
repliseq-parta v16 run 2019-03-12 19:49:08.219556
['28d0534f-a6c8-45c7-9a97-a72e377147ff', 'e0d4f71b-249b-4065-b004-879e630fc91c', '8c97269b-9169-4cee-b073-ec1e6c3600db', 'dbfd8f87-7463-4fe1-9c56-5d5612780073']
repliseq-parta v16 run 2019-03-12 19:49:00.822963
['1be98d12-146b-4f25-8e31-3b0d9a500892', '9ff5d984-62f7-4412-961c-2e71808b427d', '0e616848-97ef-4914-a055-9be922c6bcdf', '9a420689-4866-4d8f-b803-128f3206d357']


In [8]:
# Check status

# Which status to change
change_status = 'released'


change_level = STATUS_LEVEL.get(change_status, 1)
    
    
print_each = False
counter = 0
for a_type in store:
    total = len(store[a_type])
    change = 0
    matching = 0
    unusual = 0
    skipping = 0
    for raw_data in store[a_type]:
        if STATUS_LEVEL.get(raw_data['status']) > change_level:
            skipping += 1
            msg = ('{} {} ITEM HAS STATUS {} HIGHER THAN {} - SKIPPING'.format(a_type, raw_data['uuid'], raw_data['status'], change_status))
        elif STATUS_LEVEL.get(raw_data['status']) == STATUS_LEVEL.get(change_status):
            matching += 1
            msg = ('MATCHING ACCESS STATUS', a_type, raw_data['uuid'], raw_data['status'])
        elif STATUS_LEVEL.get(raw_data['status']) == 0:
            unusual += 1
            msg = ('SKIP UNUSUAL STATUS  ', a_type, raw_data['uuid'], raw_data['status'])
        else:
            change += 1
            msg = ('       CHANGE        ', a_type, raw_data['uuid'], raw_data['status'])
        if print_each:
            print(msg)
    print('{:<25} Out of {t}, {r} skipped, {m} matching, {u} unusual, and {c} needs change'.format(a_type, t=total, r=skipping, m=matching, u=unusual, c=change))
    
       

protocol                  Out of 2, 0 skipped, 2 matching, 0 unusual, and 0 needs change
file_fastq                Out of 16, 0 skipped, 16 matching, 0 unusual, and 0 needs change
workflow_run_awsem        Out of 43, 0 skipped, 20 matching, 0 unusual, and 23 needs change
biosample                 Out of 2, 0 skipped, 2 matching, 0 unusual, and 0 needs change
biosample_cell_culture    Out of 2, 0 skipped, 2 matching, 0 unusual, and 0 needs change
lab                       Out of 10, 0 skipped, 10 matching, 0 unusual, and 0 needs change
target                    Out of 3, 0 skipped, 3 matching, 0 unusual, and 0 needs change
experiment_type           Out of 1, 0 skipped, 1 matching, 0 unusual, and 0 needs change
image                     Out of 1, 0 skipped, 1 matching, 0 unusual, and 0 needs change
ontology                  Out of 2, 0 skipped, 2 matching, 0 unusual, and 0 needs change
file_format               Out of 9, 0 skipped, 9 matching, 0 unusual, and 0 needs change
antibody      

In [16]:
listem = ['experiment_hi_c',         
'experiment_set_replicate',
'experiment_chiapet',
'experiment_seq']

my_store = {i:store[i] for i in store if i in listem}
print(len(my_store))
print(my_store.keys())

4
dict_keys(['experiment_hi_c', 'experiment_set_replicate', 'experiment_chiapet', 'experiment_seq'])


In [9]:
# If you want to patch the status, change action to True
action = True

# Which status to change
change_status = 'released'


reviewed = ""
reviewed = input('Did another wrangler review this release? (y/n):')
if reviewed != 'y':
    raise KeyError('A key step is missing!')
    
print_each = False
counter = 0
for a_type in store:
    total = len(store[a_type])
    change = 0
    matching = 0
    unusual = 0
    skipping = 0
    for raw_data in store[a_type]:
        if STATUS_LEVEL.get(raw_data['status']) > change_level:
            skipping += 1
            msg = ('{} {} ITEM HAS STATUS {} HIGHER THAN {} - SKIPPING'.format(a_type, raw_data['uuid'], raw_data['status'], change_status))
        elif STATUS_LEVEL.get(raw_data['status']) == STATUS_LEVEL.get(change_status):
            matching += 1
            msg = ('MATCHING ACCESS STATUS', a_type, raw_data['uuid'], raw_data['status'])
        elif STATUS_LEVEL.get(raw_data['status']) == 0:
            unusual += 1
            msg = ('SKIP UNUSUAL STATUS  ', a_type, raw_data['uuid'], raw_data['status'])
        else:
            change += 1
            msg = ('       CHANGE        ', a_type, raw_data['uuid'], raw_data['status'])
            if action:
                patch_data = {'status': change_status}
                if change_status == 'released' and a_type in ['publication']:
                    patch_data = {'status': 'current'}
                   
                ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth) 

        if print_each:
            print(msg)
    print('{:<25} Out of {t}, {r} skipped, {m} matching, {u} unusual, and {c} UPDATED with status'.format(a_type, t=total, r=skipping, m=matching, u=unusual, c=change))

Did another wrangler review this release? (y/n):y
protocol                  Out of 2, 0 skipped, 2 matching, 0 unusual, and 0 UPDATED with status
file_fastq                Out of 16, 0 skipped, 16 matching, 0 unusual, and 0 UPDATED with status
workflow_run_awsem        Out of 43, 0 skipped, 20 matching, 0 unusual, and 23 UPDATED with status
biosample                 Out of 2, 0 skipped, 2 matching, 0 unusual, and 0 UPDATED with status
biosample_cell_culture    Out of 2, 0 skipped, 2 matching, 0 unusual, and 0 UPDATED with status
lab                       Out of 10, 0 skipped, 10 matching, 0 unusual, and 0 UPDATED with status
target                    Out of 3, 0 skipped, 3 matching, 0 unusual, and 0 UPDATED with status
experiment_type           Out of 1, 0 skipped, 1 matching, 0 unusual, and 0 UPDATED with status
image                     Out of 1, 0 skipped, 1 matching, 0 unusual, and 0 UPDATED with status
ontology                  Out of 2, 0 skipped, 2 matching, 0 unusual, and 0 UPD